In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt 
%matplotlib inline

import random as rand

import warnings
warnings.filterwarnings("ignore")

Then we load the data. We will also print a list of each columns together with its datatype, using the "type" function.

In [4]:
np.random.seed(26)
df = pd.read_csv('tmdb_5000_movies.csv') # Load in the csv file 

feature_names = df.columns
for i in range(len(feature_names)):
    print(str(i), "\t", str(feature_names[i]),"\t\t\t", str(type(df.iloc[0,i])))

0 	 budget 			 <class 'numpy.int64'>
1 	 genres 			 <class 'str'>
2 	 homepage 			 <class 'str'>
3 	 id 			 <class 'numpy.int64'>
4 	 keywords 			 <class 'str'>
5 	 original_language 			 <class 'str'>
6 	 original_title 			 <class 'str'>
7 	 overview 			 <class 'str'>
8 	 popularity 			 <class 'numpy.float64'>
9 	 production_companies 			 <class 'str'>
10 	 production_countries 			 <class 'str'>
11 	 release_date 			 <class 'str'>
12 	 revenue 			 <class 'numpy.int64'>
13 	 runtime 			 <class 'numpy.float64'>
14 	 spoken_languages 			 <class 'str'>
15 	 status 			 <class 'str'>
16 	 tagline 			 <class 'str'>
17 	 title 			 <class 'str'>
18 	 vote_average 			 <class 'numpy.float64'>
19 	 vote_count 			 <class 'numpy.int64'>
